In [1]:
import cv2
import numpy as np
import os

In [2]:
## KNN

def distance(x1,x2):
    
    return np.sqrt(((x1-x2)**2).sum())

def knnPred(trainData , testPt , k=5):
    
    distVals = []
    for i in range(trainData.shape[0]):
        
        ix = trainData[i,:-1]
        iy = trainData[i,-1]
        
        d = distance(ix , testPt)
        distVals.append((d,iy))
        
    distVals = sorted(distVals , key=lambda x:x[0])
    distVals = distVals[:k]
    
    labels = np.array(distVals)[:,-1]
    output = np.unique(labels , return_counts=True)
    
    index = output[1].argmax()
    pred = output[0][index]
    
    return pred
        
    

In [3]:
# Init camera
cap = cv2.VideoCapture(0)

#FaceDetection
faceCascades = cv2.CascadeClassifier('/Volumes/part3/haarcascades_frontalface_alt.xml')

In [4]:
skip = 0 
datasetPath = '/Volumes/part3/faceRecogDataSet/'

faceData = []  # x value for our data
labels = []     # y value for our data

classID = 0
names = {}

#Data Preparation

In [5]:
for fx in os.listdir(datasetPath):
    
    if (fx.endswith('.npy')):
        
        #Create a mapping between classID and corresponding name.
        names[classID] = fx[:-4]  
        
        
        #extract data from file
        dataItem = np.load(datasetPath+fx)
        print("Loaded "+fx)
        print(dataItem.shape)
        faceData.append(dataItem)
        
        curlabel = classID*np.ones((dataItem.shape[0],))
        # for every image captured(training data) we have a corresponding label
        
        classID+=1
        labels.append(curlabel)
        
        
    

Loaded Amit.npy
(30, 30000)
Loaded Mitul.npy
(30, 30000)
Loaded Pranav.npy
(30, 30000)


In [6]:
print(np.array(faceData).shape)
print(np.array(labels).shape)



(3, 30, 30000)
(3, 30)


In [7]:
faceDataSet = np.concatenate(faceData , axis=0)
faceLabels = np.concatenate(labels , axis=0)

In [8]:
print(faceDataSet.shape)
print(faceLabels.shape)

(90, 30000)
(90,)


In [9]:
# try

a = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])

b = np.array([
    [1],
    [2],
    [3]
])
print(a.shape)
print(b.shape)
np.concatenate((a,b),axis=1)

(3, 3)
(3, 1)


array([[1, 2, 3, 1],
       [4, 5, 6, 2],
       [7, 8, 9, 3]])

In [10]:
# clubbing the facedataset and facelabels in a single matrix to use it in KNN algorithm.

faceLabels = faceLabels.reshape((-1,1))
trainDataSet = np.concatenate((faceDataSet , faceLabels), axis = 1)
print(trainDataSet.shape)

(90, 30001)


In [11]:
# testing part

while True:
    
    ret , frame = cap.read()
    if (ret == False):
        continue
        
    faces = faceCascades.detectMultiScale(frame , 1.3 , 5)
    
    for face in faces:
        x,y,w,h = face
        offset = 10
        faceSection = frame[y-offset:y+h+offset , x-offset:x+w+offset]
        faceSection = cv2.resize(faceSection,(100,100))
        
        # Our prediction.
        predID = knnPred(trainDataSet , faceSection.flatten())
        
        
        #Display on screen name and rectangle around it.
        
        predName = names[int(predID)]
        cv2.putText(frame , predName , (x,y-10) ,  cv2.FONT_HERSHEY_PLAIN,1,(0,0,255),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        
        
    cv2.imshow("Video Capture",frame)
     
    key = cv2.waitKey(1) & 0xFF
    if (key == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()
        